<a href="https://colab.research.google.com/github/carlosgilrodrigo/TFM/blob/main/TFM_CarlosGilRodrigo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Trabajo de Fin de Máster**
Análisis de los efectos del factor de localidad en el desempeño de equipos de la Premier League.

# 1 - Introducción y objetivo

En esta actividad se utiliza un conjunto de datos con todos los partidos desde 2010 hasta 2021 en la Premier League, liga inglesa de fútbol. El objetivo final es analizar el impacto del factor de localidad en el desempeño de equipos de la Premier League, es decir, conocer la trascendencia de jugar como local. Para ello, se hará uso del proceso KDD.

# 2 - Información adicional

El conjunto de datos se puede obtener a partir del siguiente enlace: https://www.kaggle.com/datasets/pablohfreitas/all-premier-league-matches-20102021?select=df_full_premierleague.csv

Se utilizan las herramientas Panda, Numpy y Seaborn, además de las referentes a Matplotlib para poder graficar.

In [1]:
#Importación de paquetes
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
import matplotlib

Lectura del fichero de fuente de datos. Dataset "All Premier League Matches 2010-2021" con el histórico de partidos de la liga inglesa de fútbol entre los años 2010 y 2021. Cuenta con 114 columnas y ha sido extraído de la plataforma Kaggle.

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
#Lectura de datos
df = pd.read_csv('/content/drive/MyDrive/TFM/df_full_premierleague.csv')

Comprobación de los tipos de datos y del número de instancias.

In [6]:
print("DIMENSIONALIDAD DEL DATAFRAME: \n")
print(df.shape,"\n")
print("TIPOS DE DATOS: \n")
print(df.dtypes,"\n")
print("COLUMNAS: \n")
print(df.columns)

DIMENSIONALIDAD DEL DATAFRAME: 

(4070, 114) 

TIPOS DE DATOS: 

Unnamed: 0                    int64
link_match                   object
season                       object
date                         object
home_team                    object
                             ...   
sg_match_ft_acum_away       float64
goals_scored_ht_avg_away    float64
goals_conced_ht_avg_away    float64
sg_match_ht_acum_away       float64
performance_acum_away       float64
Length: 114, dtype: object 

COLUMNAS: 

Index(['Unnamed: 0', 'link_match', 'season', 'date', 'home_team', 'away_team',
       'result_full', 'result_ht', 'home_clearances', 'home_corners',
       ...
       'tackles_avg_away', 'touches_avg_away', 'yellow_cards_avg_away',
       'goals_scored_ft_avg_away', 'goals_conced_ft_avg_away',
       'sg_match_ft_acum_away', 'goals_scored_ht_avg_away',
       'goals_conced_ht_avg_away', 'sg_match_ht_acum_away',
       'performance_acum_away'],
      dtype='object', length=114)


Comprobación de las variables numéricas del conjunto de datos.

In [7]:
df_numeric = df.select_dtypes(include = [np.number])
numeric_cols = df_numeric.columns.values
print(numeric_cols)

['Unnamed: 0' 'home_clearances' 'home_corners' 'home_fouls_conceded'
 'home_offsides' 'home_passes' 'home_possession' 'home_red_cards'
 'home_shots' 'home_shots_on_target' 'home_tackles' 'home_touches'
 'home_yellow_cards' 'away_clearances' 'away_corners'
 'away_fouls_conceded' 'away_offsides' 'away_passes' 'away_possession'
 'away_red_cards' 'away_shots' 'away_shots_on_target' 'away_tackles'
 'away_touches' 'away_yellow_cards' 'goal_home_ft' 'goal_away_ft'
 'sg_match_ft' 'goal_home_ht' 'goal_away_ht' 'sg_match_ht'
 'clearances_avg_H' 'corners_avg_H' 'fouls_conceded_avg_H'
 'offsides_avg_H' 'passes_avg_H' 'possession_avg_H' 'red_cards_avg_H'
 'shots_avg_H' 'shots_on_target_avg_H' 'tackles_avg_H' 'touches_avg_H'
 'yellow_cards_avg_H' 'goals_scored_ft_avg_H' 'goals_conced_ft_avg_H'
 'sg_match_ft_acum_H' 'goals_scored_ht_avg_H' 'goals_conced_ht_avg_H'
 'sg_match_ht_acum_H' 'performance_acum_H' 'clearances_avg_A'
 'corners_avg_A' 'fouls_conceded_avg_A' 'offsides_avg_A' 'passes_avg_A'
 'pos

Comprobación de las variables no numéricas del conjunto de datos.

In [8]:
df_non_numeric = df.select_dtypes(exclude = [np.number])
non_numeric_cols = df_non_numeric.columns.values
print(non_numeric_cols)

['link_match' 'season' 'date' 'home_team' 'away_team' 'result_full'
 'result_ht']


# 3 - Limpieza de datos

En el proceso KDD, tras la selección del conjunto de datos, hay que limpiar dichos datos en lo que llamamos Data Cleaning. La limpieza de datos pertenece al preprocesado de datos y se basa en la transformación de los datos en bruto que se recogieron, en una forma que pueda ser utilizada en el modelado. Normalmente, las técnicas se basan en adición, eliminación o tranformación de los datos.

3.1 - Tratamiento de missing values

Tratamiento de valores desconocidos o perdidos mediante la eliminación de filas con valores perdidos, la imputación de valores perdidos, el uso de algoritmos que admitan valores perdidos o la predicción de valores perdidos.

Los valores nulos o perdidos no proporcionan información para un modelo. Por ello, y teniendo en cuenta el número de atributos diferentes que tiene el conjunto de datos, se van a eliminar las filas que tengan más del 20% de atributos con datos faltantes, ya que dichas filas no estarían lo suficientemente completas como para tenerlas en cuenta.

Antes de nada, analizamos el porcentaje de datos faltantes en cada columna.

In [9]:
#Porcentaje de missing values en cada columna
for col in df.columns:
    pct_missing = np.mean(df[col].isnull())
    print('{} - {}%'.format(col, round(pct_missing*100)))

Unnamed: 0 - 0%
link_match - 0%
season - 0%
date - 0%
home_team - 0%
away_team - 0%
result_full - 0%
result_ht - 0%
home_clearances - 0%
home_corners - 0%
home_fouls_conceded - 0%
home_offsides - 0%
home_passes - 0%
home_possession - 0%
home_red_cards - 0%
home_shots - 0%
home_shots_on_target - 0%
home_tackles - 0%
home_touches - 0%
home_yellow_cards - 0%
away_clearances - 0%
away_corners - 0%
away_fouls_conceded - 0%
away_offsides - 0%
away_passes - 0%
away_possession - 0%
away_red_cards - 0%
away_shots - 0%
away_shots_on_target - 0%
away_tackles - 0%
away_touches - 0%
away_yellow_cards - 0%
goal_home_ft - 0%
goal_away_ft - 0%
sg_match_ft - 0%
goal_home_ht - 0%
goal_away_ht - 0%
sg_match_ht - 0%
clearances_avg_H - 5%
corners_avg_H - 5%
fouls_conceded_avg_H - 5%
offsides_avg_H - 5%
passes_avg_H - 5%
possession_avg_H - 5%
red_cards_avg_H - 5%
shots_avg_H - 5%
shots_on_target_avg_H - 5%
tackles_avg_H - 5%
touches_avg_H - 5%
yellow_cards_avg_H - 5%
goals_scored_ft_avg_H - 5%
goals_conced_